In [1]:
import os
import sys
import pandas as pd
import time

# Import from the parent directory
sys.path.append(os.path.join(os.getcwd(), '..'))
from teamwork import teamwork_v2 as tw

In [2]:
start_time = time.perf_counter()

# discharges_test_file = '../data/discharges_test.csv'
# notes_test_file = '../data/notes_test.csv'

discharges_test_file = '../data/discharges_w_disposition_abdul_test.csv'
notes_test_file = '../data/notes_w_disposition_abdul_test.csv'

discharge_table = pd.read_csv(discharges_test_file, parse_dates=['arrive_date','discharge_date'])
notes_table = pd.read_csv(notes_test_file, parse_dates=['date'])
notes_table.rename(columns = {'discharge_id':'id'}, inplace = True)

# match in admission datetime indexing on visit id from discharge table
notes_table = notes_table.merge(discharge_table, on='id', how='right')

corpus = tw.TeamworkCorpus(notes_table)
stop_time = time.perf_counter()
print(f"It took {stop_time - start_time} seconds or {(stop_time - start_time) / 60} minutes to create teamwork corpus")   

print(len(corpus.team_experience_dict))

/home/jovyan/work/notebooks/../teamwork/teamwork_v2.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  notes_df[NORM_ADMISSION_DATE] = notes_df[admit_date].astype('datetime64[D]')
/home/jovyan/work/notebooks/../teamwork/teamwork_v2.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  notes_df[NORM_NOTE_DATE] = notes_df[note_date].astype('datetime64[D]')
/home/jovyan/work/notebooks/../teamwork/teamwork_v2.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See

teamwork_delta: 90
teamwork_delta: 2020-03-09 00:00:00


TypeError: Addition/subtraction of integers and integer-arrays with Timestamp is no longer supported.  Instead of adding/subtracting `n`, use `n * obj.freq`